In [1]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
import pandas as pd

In [2]:
urls = ["https://goias.gov.br/social/perguntas-e-respostas-frequentes-aprendiz-do-futuro/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-dignidade/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-maes-de-goias/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-goias-por-elas/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-sistema-socioeducativo/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-auxilios-agua-e-energia-pao-e-leite/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-carteira-de-identificacao-do-autista2/",
        "https://goias.gov.br/social/bolsa-familia/",
        "https://goias.gov.br/social/cadastro-unico-cadunico/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passaporte-intermunicipal-da-pessoa-idosa/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passe-livre-intermunicipal-da-pessoa-com-deficiencia/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-centro-de-referencia-estadual-da-igualdade-crei/"]

inicio = """Categoria  

Perguntas e Respostas Frequentes"""

fim = """


 











 









 













 

 





 Governo na palma da mão"""

complemento_texto = "Fui vítima de"

In [3]:
textos_brutos = []
textos_conteudo = []
textos_process = []

for url in urls:
    textos = WebBaseLoader(url)
    textos_brutos = textos.load()
    textos_conteudo.append(textos_brutos[0].page_content)

In [4]:
tamanho_inicio = len(inicio)
tamanho_final = len(fim)
tamanho_complemento = len(complemento_texto)

for respostas_ind in textos_conteudo:
    for j in range(len(respostas_ind) - tamanho_inicio):
        if inicio in respostas_ind[j:j + tamanho_inicio]:
            começo_frase = j + tamanho_inicio
        elif fim in respostas_ind[j:j+tamanho_final]:
            textos_process.append(respostas_ind[começo_frase + 6 : j])

In [19]:
textos_split = []
questions = []
perguntas = []
answers = []
respostas = []

In [20]:
for texto in textos_process:
    for i, caractere in enumerate(texto):
        aux = i 
        if "?" in caractere:
            pergunta = ""
            interrogacao = i
            cont = 1
            contador = 0
            while contador != 1:
                if texto[i - cont] == "." or texto[i - cont] == ";" or texto[i - cont] == "\n" or (i - cont) == 0 :
                    j = i - cont
                    contador = 1
                cont = cont + 1
            cont = 0
            while j + cont <= i:
                pergunta = pergunta + texto[j + cont]
                cont = cont + 1
            if pergunta == ". O que devo fazer?":
                aux = i
                while complemento_texto not in texto[aux : aux + len(complemento_texto)]:
                    aux = aux - 1
                pergunta = texto[aux : i]
            questions.append(pergunta)

In [21]:
for i, question in enumerate(questions):
    if question == 'O que é o CREI?':
        del questions[i]
questions = [caractere.replace("\n", "") for caractere in questions]

In [22]:
cont = 0
while cont < len(questions):
    for texto in textos_process:
        for i, caractere in enumerate(texto):
            if cont < len(questions) - 1:
                if questions[cont] in texto[i: i + len(questions[cont])]:
                    inicio_respostas = i + len(questions[cont]) + 1
                    if cont + 1 < len(questions):
                        j = inicio_respostas
                        for j in range (len(texto) - len(questions[cont + 1])):
                            if questions[cont + 1] in texto[j: j + len(questions[cont + 1])]:
                                answers.append(texto[inicio_respostas : j - 1])
                                cont = cont + 1
                                break
                            if questions[cont + 1] not in texto[j: j + len(questions[cont + 1])] and j + len(questions[cont + 1]) == len(texto) - 1:
                                answers.append(texto[i + len(questions[cont]) :])
                                cont = cont + 1
                                break 
            elif cont == len(questions) - 1:
                answers.append(texto[i:])
                cont = cont + 1

In [23]:
answers = [caractere.replace("\n", "") for caractere in answers]
answers = [caractere.replace("– ", "") for caractere in answers]

In [24]:
all_data =[]

In [25]:
for item in range(len(questions)):
    question = questions[item]
    answer = answers[item]
    all_data.append({'question': question, 'answer': answer})

In [26]:
data_df = pd.DataFrame(all_data)
data_df.to_csv('Data/rag_doc.csv', index=False)